<h1><center>Laboratorio 7: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Natalia Abarca G.
- Nombre de alumno 2: César Véliz R.


### **Link de repositorio de GitHub:**`` https://github.com/CVelizR/evaluaciones-mds7202 ``

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- Fecha de entrega: TBD
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [27]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import r2_score
from sklearn.linear_model import ElasticNet


#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# UMAP
!pip install umap-learn

https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [2]:
df_players = pd.read_csv('stats_players.csv')
df_players

Name          Nationality National_Position Club_Position  \
0      Cristiano Ronaldo             Portugal                LS            LW   
1           Lionel Messi            Argentina                RW            RW   
2                 Neymar               Brazil                LW            LW   
3            Luis Suárez              Uruguay                LS            ST   
4           Manuel Neuer              Germany                GK            GK   
...                  ...                  ...               ...           ...   
17583        Adam Dunbar  Republic of Ireland               NaN           Sub   
17584       Dylan McGoey  Republic of Ireland               NaN           Sub   
17585    Tommy Ouldridge              England               NaN           Res   
17586         Mark Foden             Scotland               NaN           Sub   
17587   Barry Richardson              England               NaN           Sub   

       Height  Weight Preffered_Foot  Age        Work_Rate  Weak_foot  ...  \
0         185      80          Right   32       High / Low          4  ...   
1         170      72           Left   29  Medium / Medium          4  ...   
2         174      68          Right   25    High / Medium          5  ...   
3         182      85          Right   30    High / Medium          4  ...   
4         193      92          Right   31  Medium / Medium          4  ...   
...       ...     ...            ...  ...              ...        ...  ...   
17583     183      82          Right   19  Medium / Medium          1  ...   
17584     185      80          Right   19  Medium / Medium          2  ...   
17585     173      61          Right   18    High / Medium          2  ...   
17586     180      80          Right   21  Medium / Medium          3  ...   
17587     185      77          Right   47  Medium / Medium          2  ...   

       Agility  Jumping  Heading  Shot_Power  Finishing  Long_Shots  Curve  \
0           90       95       85          92         93          90     81   
1           90       68       71          85         95          88     89   
2           96       61       62          78         89          77     79   
3           86       69       77          87         94          86     86   
4           52       78       25          25         13          16     14   
...        ...      ...      ...         ...        ...         ...    ...   
17583       27       56       14          16         13          13     11   
17584       28       53       12          17         12          11     12   
17585       54       61       41          44         28          42     35   
17586       34       48       15          23         14          12     13   
17587       38       51       12          13         11          16     12   

       Freekick_Accuracy  Penalties  Volleys  
0                     76         85       88  
1                     90         74       85  
2                     84         81       83  
3                     84         85       88  
4                     11         47       11  
...                  ...        ...      ...  
17583                 13         15       12  
17584                 13         16       12  
17585                 36         42       37  
17586                 12         24       12  
17587                 11         22       12  

[17588 rows x 39 columns]

## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [3]:
df1 = df_players.copy()
df1["label"] = df1["National_Position"]
df1.loc[~df1["label"].isnull(), "label"] = 1
df1.loc[df1["label"].isnull(), "label"] = 0
df1

Name          Nationality National_Position Club_Position  \
0      Cristiano Ronaldo             Portugal                LS            LW   
1           Lionel Messi            Argentina                RW            RW   
2                 Neymar               Brazil                LW            LW   
3            Luis Suárez              Uruguay                LS            ST   
4           Manuel Neuer              Germany                GK            GK   
...                  ...                  ...               ...           ...   
17583        Adam Dunbar  Republic of Ireland               NaN           Sub   
17584       Dylan McGoey  Republic of Ireland               NaN           Sub   
17585    Tommy Ouldridge              England               NaN           Res   
17586         Mark Foden             Scotland               NaN           Sub   
17587   Barry Richardson              England               NaN           Sub   

       Height  Weight Preffered_Foot  Age        Work_Rate  Weak_foot  ...  \
0         185      80          Right   32       High / Low          4  ...   
1         170      72           Left   29  Medium / Medium          4  ...   
2         174      68          Right   25    High / Medium          5  ...   
3         182      85          Right   30    High / Medium          4  ...   
4         193      92          Right   31  Medium / Medium          4  ...   
...       ...     ...            ...  ...              ...        ...  ...   
17583     183      82          Right   19  Medium / Medium          1  ...   
17584     185      80          Right   19  Medium / Medium          2  ...   
17585     173      61          Right   18    High / Medium          2  ...   
17586     180      80          Right   21  Medium / Medium          3  ...   
17587     185      77          Right   47  Medium / Medium          2  ...   

       Jumping  Heading  Shot_Power  Finishing  Long_Shots  Curve  \
0           95       85          92         93          90     81   
1           68       71          85         95          88     89   
2           61       62          78         89          77     79   
3           69       77          87         94          86     86   
4           78       25          25         13          16     14   
...        ...      ...         ...        ...         ...    ...   
17583       56       14          16         13          13     11   
17584       53       12          17         12          11     12   
17585       61       41          44         28          42     35   
17586       48       15          23         14          12     13   
17587       51       12          13         11          16     12   

       Freekick_Accuracy  Penalties  Volleys  label  
0                     76         85       88      1  
1                     90         74       85      1  
2                     84         81       83      1  
3                     84         85       88      1  
4                     11         47       11      1  
...                  ...        ...      ...    ...  
17583                 13         15       12      0  
17584                 13         16       12      0  
17585                 36         42       37      0  
17586                 12         24       12      0  
17587                 11         22       12      0  

[17588 rows x 40 columns]

Notar que la clase 0 corresponde a los jugadores no seleccionados y la clase 1 los que sí lo son:

In [4]:
print("Cantidad de filas para clase 0: "+str(df1["label"].value_counts()[0]))

print("Cantidad de filas para clase 1: "+str(df1["label"].value_counts()[1]))

Cantidad de filas para clase 0: 16513
Cantidad de filas para clase 1: 1075


Dado los resultados obtenidos, se aprecia un desbalance de clases. Es decir, se tiene una cantidad bastante mayor de filas pertenecientes a la clase 0 en comparación con filas pertenecientes a la clase 1.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [X] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [X] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [X] Crear 3 ``pipelines`` con diferentes clasificadores.
- [X] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [X] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

Para aplicar ColumnTransformer, todas las variables que tengan una distribución aproximadamente normal se les aplicará StandardScaler con el fin de que tengan media 0 y desviación estándar unitaria. Por otro lado, aquellas variables que tengan "skewness" se les aplicará PowerTransform. Además, aquellas que estén relativamente uniformemente distribuidas se les aplicará MinMaxScaler. Finalmente, las variables dummy y categóricas se les aplicará OneHotEncoder. Es importante notar que no se utilizarán todas las características pues algunas tienen alta correlación entre sí.

Dicho esto, el ColumnTransformer queda como a continuación:

In [5]:
ct= ColumnTransformer([
    ("StandardScaler",StandardScaler(),["Height","Weight","Aggression","Reactions","Vision","Composure","Crossing","Short_Pass","Long_Pass","Acceleration","Speed","Stamina","Strength","Balance","Agility","Jumping","Freekick_Accuracy"]),
    ("PowerTransform",PowerTransformer(),['Age',"Ball_Control","Dribbling","Marking","Sliding_Tackle","Standing_Tackle","Interceptions","Heading","Shot_Power"]),
    ("MinMaxScaler",MinMaxScaler(),["Finishing","Long_Shots","Curve","Penalties","Volleys"]),
    ("OneHot",OneHotEncoder(handle_unknown="ignore"),["Preffered_Foot","Work_Rate","Skill_Moves"])
])

Luego, para crear los Pipeline, se utilizó la guía. Dada la naturaleza del problema, utilizaremos 3 modelos de clasificación, los cuales serán: DecisionTreeClassifier, SupportVectorClassifier y KNeighbourClassifier.

In [6]:
pipe1 = Pipeline([("ColumnTransform", ct),
                     ("DTC", DecisionTreeClassifier(random_state=0))])

In [7]:
from sklearn.svm import SVC
pipe2 = Pipeline([("ColumnTransform", ct),
                     ("SVC", SVC())])

In [8]:
from sklearn.neighbors import KNeighborsClassifier
pipe3 = Pipeline([("ColumnTransform", ct),
                     ("KNeighbour", KNeighborsClassifier())])

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [X] Separar el conjunto de datos en entrenamiento y de prueba.
- [X] Entrenar los `pipelines`.
- [X] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [9]:
df = df1.copy()
X = df.drop(["Name", "Nationality", "label"], 1)
Y = df1["label"].to_list()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=24)

C:\Users\Cesar\AppData\Local\Temp/ipykernel_15716/2073846275.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = df.drop(["Name", "Nationality", "label"], 1)


In [10]:
# Verificar proporciones
train = sum(y_train)/ len(y_train) 
test = sum(y_test) / len(y_test) 
print(f"Proporcion labels 1 en train: {train}")
print(f"Proporcion labels 1 en test: {test}")

Proporcion labels 1 en train: 0.061164811956786615
Proporcion labels 1 en test: 0.06101951866590866


In [11]:
# Entrenamiento pipeline 1
pipe1.fit(X_train, y_train)
y_pred1 = pipe1.predict(X_test)
print(classification_report(y_test, y_pred1))

              precision    recall  f1-score   support

           0       0.95      0.94      0.94      4955
           1       0.19      0.22      0.20       322

    accuracy                           0.90      5277
   macro avg       0.57      0.58      0.57      5277
weighted avg       0.90      0.90      0.90      5277



In [12]:
# Entrenamiento pipeline 2
pipe2.fit(X_train, y_train)
y_pred2 = pipe2.predict(X_test)
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4955
           1       1.00      0.00      0.01       322

    accuracy                           0.94      5277
   macro avg       0.97      0.50      0.49      5277
weighted avg       0.94      0.94      0.91      5277



In [13]:
# Entrenamiento pipeline 3
pipe3.fit(X_train, y_train)
y_pred3 = pipe3.predict(X_test)
print(classification_report(y_test, y_pred3))

              precision    recall  f1-score   support

           0       0.94      0.99      0.97      4955
           1       0.42      0.09      0.15       322

    accuracy                           0.94      5277
   macro avg       0.68      0.54      0.56      5277
weighted avg       0.91      0.94      0.92      5277



**Respuesta**:

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?

Accuracy indica la cantidad de predicciones correctas que se realizaron. Precision indica la proporción de ejemplos correctamente predichos a cierta clase con respecto a todos los que se predijeron como dicha clase y recall corresponde a la proporción de ejemplos correctamente predichos para cierta clase con respecto a la cantidad de ejemplos que efectivamente deberían haber sido clasificados para dicha clase. De los resultados obtenidos, se aprecia un alto accuracy, pero esto se debe al desbalance de clases, lo cual hace que nuestro modelo se sobreajuste a la clase 0. Esto se aprecia en las métricas de precisión y recall, debido a que son notoriamente más bajas para los datos pertenecientes a la clase 1.


- ¿Podrían mejorarse los resultados?, ¿Cómo?

Los resultados podrían mejorarse con técnicas de preprocesamiento. Aplicar tipos de transformaciones distintos a los implementados, reducir atributos redundantes o agregar características nuevas. Por otro lado, se nota una clara tendencia a predecir bien los datos de clase 0 en comparación con aquellos de clase 1, esto debido al desbalance. Contar con una mayor cantidad de datos que pertenezcan a la clase 1 podría mejorar los resultados (también aplicar técnicas de oversampling/subsampling).

- ¿Influye la cantidad de ejemplos por clase?

Sí, debido a que si se tienen clases muy desbalanceadas, el modelo posiblemente se sobreajuste a la clase que predomine cuando se entrene.





## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [X] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [X] Cuente cuantos por clase quedan.
- [X] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [X] Comente los resultados obtenidos.

**Respuesta:**

In [14]:
# Mapeo de Club Position
df2 = df_players.copy()
df2["Club_Position"] = df2["Club_Position"].map({'ST': "ataque", 'CF': "ataque",
                                'RW': "central_ataque", 'CAM': "central_ataque", 'LW': "central_ataque", 
                                'RM': "central", 'CM': "central", 'LM': "central", 
                                'RWB': "central_defensa", 'CDM': "central_defensa", 'LWB': "central_defensa",
                                'RB': "defensa", 'CB': "defensa", 'LB': "defensa",
                                'GK': "arquero"})
df2 = df2.dropna(subset="Club_Position", axis=0)

In [15]:
# Contar valores por clase
df2["Club_Position"].value_counts()

defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: Club_Position, dtype: int64

In [16]:
# Crear columnas con etiquetas
df2["Labels"] = df2["Club_Position"].map({"defensa": 0,
                                         "central": 1,
                                         "arquero": 2,
                                         "central_ataque": 3,
                                         "ataque": 4,
                                         "central_defensa": 5})

In [17]:
ct2= ColumnTransformer([
    ("StandardScaler",StandardScaler(),["Height","Weight","Aggression","Reactions","Vision","Composure","Crossing","Short_Pass","Long_Pass","Acceleration","Speed","Stamina","Strength","Balance","Agility","Jumping","Freekick_Accuracy"]),
    ("PowerTransform",PowerTransformer(),['Age',"Ball_Control","Dribbling","Marking","Sliding_Tackle","Standing_Tackle","Interceptions","Heading","Shot_Power"]),
    ("MinMaxScaler",MinMaxScaler(),["Finishing","Long_Shots","Curve","Penalties","Volleys"]),
    ("OneHot",OneHotEncoder(handle_unknown="ignore"),["Preffered_Foot","Work_Rate","Skill_Moves"])
])

pipe2 = Pipeline([("ColumnTransform", ct2),
                     ("SVC", SVC())])

In [18]:
# Separación de train-test
df = df2.copy()
X = df.drop(["Name", "Nationality", "National_Position", "Labels", "Club_Position"], 1)
Y = df2["Labels"].to_list()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

C:\Users\Cesar\AppData\Local\Temp/ipykernel_15716/2966790828.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = df.drop(["Name", "Nationality", "National_Position", "Labels", "Club_Position"], 1)


In [19]:
# Propoción de clases en train y test
u_train, c_train = np.unique(y_train, return_counts=True)
u_test, c_test = np.unique(y_test, return_counts=True)

train = c_train / len(y_train)
test = c_test/ len(y_test)
print(f"Proporcion labels en train: {train}")
print(f"Proporcion labels en test: {test}")

Proporcion labels en train: [0.2992383  0.22488212 0.16503446 0.15161407 0.10591222 0.05331882]
Proporcion labels en test: [0.30033841 0.2428088  0.14974619 0.13790186 0.11675127 0.05245347]


In [20]:
# Entrenamiento y predicción
pipe2.fit(X_train, y_train)
y_pred2 = pipe2.predict(X_test)
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.80      0.97      0.88       355
           1       0.60      0.66      0.63       287
           2       1.00      1.00      1.00       177
           3       0.51      0.27      0.35       163
           4       0.81      0.82      0.81       138
           5       0.46      0.19      0.27        62

    accuracy                           0.75      1182
   macro avg       0.69      0.65      0.66      1182
weighted avg       0.72      0.75      0.72      1182



De los resultados obtenidos, se aprecia que para ciertas clases se logra predecir de forma decente, pero para otras no. Esto se puede deber al desbalance de clases que existe en el dataset y a la cantidad de datos que se tiene para entrenar. 

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [X] Explique en que consiste la métrica $R^2$
- [X] Generar un pipeline para la regresión.
- [X] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [X] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

La métrica **$R^2$** consiste en un score o puntaje que representa la proporción de varianza de los valores predichos por el modelo con respecto a la las etiquetas reales. Dicho esto, el mejor puntaje posible es 1 y si el modelo es extremadamente malo, pueden obtenerse resultados negativos.

In [21]:
salarios = pd.read_csv("salarios.csv")
salarios = salarios.drop("Unnamed: 0", 1)
salarios = salarios.rename(columns={"Player": "Name"})

C:\Users\Cesar\AppData\Local\Temp/ipykernel_15716/28356162.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  salarios = salarios.drop("Unnamed: 0", 1)


In [22]:
df3 = pd.merge(left=df_players,
                    right=salarios,
                    on="Name",
                    sort=True,
                    how="outer")
df3 = df3.dropna(subset="Weekly Salary", axis=0)
df3 = df3.dropna(subset="Height", axis=0)

In [23]:
df3.describe()

Height       Weight          Age    Weak_foot  Skill_Moves  \
count  1866.000000  1866.000000  1866.000000  1866.000000  1866.000000   
mean    182.366559    76.266345    24.930868     3.052519     2.457663   
std       6.546156     7.033456     4.131101     0.680976     0.856815   
min     162.000000    48.000000    17.000000     1.000000     1.000000   
25%     178.000000    72.000000    22.000000     3.000000     2.000000   
50%     183.000000    76.000000    25.000000     3.000000     2.000000   
75%     187.000000    81.000000    28.000000     3.000000     3.000000   
max     201.000000    99.000000    39.000000     5.000000     5.000000   

       Ball_Control    Dribbling      Marking  Sliding_Tackle  \
count   1866.000000  1866.000000  1866.000000     1866.000000   
mean      65.370311    61.552519    48.619507       50.511790   
std       18.225297    20.614518    24.436182       24.733586   
min       10.000000     5.000000     5.000000        8.000000   
25%       62.000000    56.000000    23.000000       24.000000   
50%       71.000000    68.000000    56.000000       59.000000   
75%       77.000000    75.000000    71.000000       72.000000   
max       95.000000    97.000000    92.000000       91.000000   

       Standing_Tackle  ...      Jumping      Heading   Shot_Power  \
count       1866.00000  ...  1866.000000  1866.000000  1866.000000   
mean          53.00000  ...    68.071811    56.762594    61.620043   
std           24.68378  ...    11.157666    19.228412    18.777998   
min            9.00000  ...    30.000000     8.000000    11.000000   
25%           28.00000  ...    61.000000    49.000000    52.000000   
50%           63.00000  ...    69.000000    61.000000    68.000000   
75%           74.00000  ...    75.000000    70.000000    75.750000   
max           92.00000  ...    95.000000    93.000000    92.000000   

         Finishing   Long_Shots        Curve  Freekick_Accuracy    Penalties  \
count  1866.000000  1866.000000  1866.000000        1866.000000  1866.000000   
mean     49.897642    53.172026    53.268489          47.702036    52.474812   
std      21.645839    20.897269    20.428841          19.502833    16.936046   
min       5.000000     5.000000     6.000000           7.000000    11.000000   
25%      32.000000    38.000000    40.000000          33.250000    42.000000   
50%      54.000000    59.000000    58.000000          48.000000    54.000000   
75%      68.000000    69.000000    69.000000          64.000000    65.000000   
max      95.000000    91.000000    90.000000          91.000000    92.000000   

           Volleys  Weekly Salary  
count  1866.000000   1.866000e+03  
mean     48.392283   5.363130e+04  
std      20.205789   8.193256e+04  
min       5.000000   6.200000e+02  
25%      32.250000   1.413700e+04  
50%      51.000000   2.940400e+04  
75%      65.000000   6.502900e+04  
max      91.000000   1.538905e+06  

[8 rows x 34 columns]

In [24]:
ct3= ColumnTransformer([
    ("StandardScaler",StandardScaler(),["Height","Weight","Aggression","Reactions","Vision","Composure","Crossing","Short_Pass","Long_Pass","Acceleration","Speed","Stamina","Strength","Balance","Agility","Jumping","Freekick_Accuracy"]),
    ("PowerTransform",PowerTransformer(),['Age',"Ball_Control","Dribbling","Marking","Sliding_Tackle","Standing_Tackle","Interceptions","Heading","Shot_Power"]),
    ("MinMaxScaler",MinMaxScaler(),["Finishing","Long_Shots","Curve","Penalties","Volleys"]),
    ("OneHot",OneHotEncoder(handle_unknown="ignore"),["Nationality","Preffered_Foot","Work_Rate","Skill_Moves"])
])

In [25]:
df = df3.copy()
X = df.drop(["Name", "Weekly Salary"], 1)
Y = df3["Weekly Salary"].to_list()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=24)

C:\Users\Cesar\AppData\Local\Temp/ipykernel_15716/2664637799.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = df.drop(["Name", "Weekly Salary"], 1)


Debido a que este es un problema de regresión (es decir, predecir un valor continuo) y siguiendo las intrucciones de la guía se decidió utilizar el modelo de ElasticNet

In [28]:
# Club_position, weak_foot, preffered foot, Work_Rate, stamina
elastic_pipe =Pipeline([('ColumnTransform', ct3) , 
                       ('Regresor', ElasticNet(alpha=0.01))])

elastic_pipe.fit(X_train, y_train)

elastic_y_pred = elastic_pipe.predict(X_test)

print(f" R² score: {r2_score(y_test, elastic_y_pred)}")                       

 R² score: 0.36680824803810697


**Respuesta**:

De los resultados obtenidos, se obtuvo un score de aproximadamente 0,366. Este score como tal no es muy bueno, pero se debe tener en cuenta que la base de datos que se utilizó para entrenar no tiene muchas filas, lo cual perjudica a la hora de entrenar el modelo. Por otro lado, quizás métodos de creación de variables, como lo puede ser aplicar ciertos estadísticos, podrían ayudar a obtener mejores resultados.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>